In [1]:
import pandas as pd
from sklearn import preprocessing
file_name = "out1.csv"

df = pd.read_csv(file_name)
df.tail(5)  # print first 5 rows of the dataframe

,PERSONEN_NR,BUCHUNGS_TAG,HAENDLER,BETRAG_UMSATZ
648967,600,05.02.2020,Stadtwerke Kiel Aktiengesellschaft,77.66
648968,600,26.09.2020,Deutsche Welthungerhilfe eV,9.04
648969,600,28.11.2020,AMAZON EU S.A R.L. AMAZON EU S.A R.L.,36.48
648970,600,22.12.2020,Vodafone GmbH,79.97
648971,600,10.08.2020,KONDITOREI JUNGE GM,1.18


In [2]:
df.columns
le = preprocessing.LabelEncoder()
df['HAENDLER'] = le.fit_transform(df.HAENDLER.values)
df.head(5)

,PERSONEN_NR,BUCHUNGS_TAG,HAENDLER,BETRAG_UMSATZ
0,1,24.09.2020,24,29.37
1,1,11.11.2020,111,4.70
2,1,03.12.2020,72,13.42
3,1,13.04.2020,3,25.87
4,1,20.04.2020,82,81.00


In [3]:
df.HAENDLER = df.HAENDLER.astype(str)



In [4]:
df3 = df.groupby(['BUCHUNGS_TAG','PERSONEN_NR'],as_index=False).agg({ 'BETRAG_UMSATZ': 'sum','HAENDLER':','.join})
df3

,BUCHUNGS_TAG,PERSONEN_NR,BETRAG_UMSATZ,HAENDLER
0,01.01.2020,1,66.81,"64,39"
1,01.01.2020,2,412.52,"144,191,35,28,153,190,77"
2,01.01.2020,3,104.16,"91,133,62"
3,01.01.2020,4,36.82,"148,134"
4,01.01.2020,5,146.39,"105,24,97"
...,...,...,...,...
207785,31.10.2020,596,177.39,"89,109,55"
207786,31.10.2020,597,65.69,"15,3,68"
207787,31.10.2020,598,206.70,"190,116,153"
207788,31.10.2020,599,39.57,106


In [5]:

df4 = pd.concat([df3, df3['HAENDLER'].str.get_dummies(sep=',')], axis = 1).drop('HAENDLER', 1)
df4['BUCHUNGS_TAG'] = pd.to_datetime(df4['BUCHUNGS_TAG'])
df4

,BUCHUNGS_TAG,PERSONEN_NR,BETRAG_UMSATZ,0,1,10,100,101,102,103,...,90,91,92,93,94,95,96,97,98,99
0,2020-01-01,1,66.81,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-01,2,412.52,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-01,3,104.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2020-01-01,4,36.82,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-01,5,146.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207785,2020-10-31,596,177.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207786,2020-10-31,597,65.69,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207787,2020-10-31,598,206.70,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207788,2020-10-31,599,39.57,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
date_first = df4['BUCHUNGS_TAG'].min()  
date_last = df4['BUCHUNGS_TAG'].max()
print(date_first,date_last)

2020-01-01 00:00:00 2020-12-30 00:00:00


In [7]:

res = df4
mux = pd.MultiIndex.from_product([pd.date_range(date_first, date_last, freq='d'),res['PERSONEN_NR'].unique()], names=['BUCHUNGS_TAG', 'PERSONEN_NR'])

result = res.set_index(['BUCHUNGS_TAG', 'PERSONEN_NR']).reindex(mux, fill_value=0).reset_index()
result

,BUCHUNGS_TAG,PERSONEN_NR,BETRAG_UMSATZ,0,1,10,100,101,102,103,...,90,91,92,93,94,95,96,97,98,99
0,2020-01-01,1,66.81,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-01-01,2,412.52,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-01,3,104.16,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2020-01-01,4,36.82,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-01-01,5,146.39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218995,2020-12-30,508,135.11,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
218996,2020-12-30,538,231.94,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218997,2020-12-30,555,11.89,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
218998,2020-12-30,574,288.63,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
targets = [17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7,17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7,17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7,17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7,17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7,17,17,17,17,17,17,17,17,17,17,5,5,5,8,8,8,10,10,10,13,13,13,3,3,3,15,15,15,4,4,4,6,6,6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,9,9,9,9,9,9,12,12,12,12,12,12,14,14,14,14,14,14,16,16,16,16,16,16,18,18,18,18,18,18,19,19,19,19,19,19,6,11,7]
targets_multi = [['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['EF'],['EF'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['E'],['A,C,E'],['A,C,E'],['A,C,E'],['A,D,E,G'],['A,D,E,G'],['A,D,E,G'],['B,C,E,F'],['B,C,E,F'],['B,C,E,F'],['B,E,F'],['B,E,F'],['B,E,F'],['A,B,C,E,G'],['A,B,C,E,G'],['A,B,C,E,G'],['C,E,G'],['C,E,G'],['C,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,B,D,E,G'],['A,C,E,F'],['A,C,E,F'],['A,C,E,F'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,E,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,B,C,D,F,G'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['A,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['B,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['C,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['D,E'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,F'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['E,G'],['A,C,E,F'],['B,D,E,G'],['A,C,E,F,G']]


In [27]:
def remove(x):
    return x.replace("'", "")


A,C,E


In [11]:
finaltxt = ""
for i in range (400,600):
    userdf = result.loc[result['PERSONEN_NR']==i+1]
    channel = ""
    for j in range (2,197):
        x = userdf.iloc[:,j].to_string(header = False,index = False).split('\n')
        vals = [','.join(ele.split()) for ele in x]
        channel = channel + ','.join(vals) +":"
        
    userts = channel+str(targets_multi[i])
    finaltxt = finaltxt  + userts + "\n"
    
    
with open(r'C:\Users\parham\Desktop\tsai\Final\ts_test200_multi.ts', "w") as text_file:
    text_file.write(finaltxt)
   
    

In [10]:
finaltxt = ""
for i in range (400):
    userdf = result.loc[result['PERSONEN_NR']==i+1]
    channel = ""
    for j in range (2,197):
        x = userdf.iloc[:,j].to_string(header = False,index = False).split('\n')
        vals = [','.join(ele.split()) for ele in x]
        channel = channel + ','.join(vals) +":"
        
    userts = channel+str(targets_multi[i])
    finaltxt = finaltxt  + userts + "\n"
    
    
with open(r'C:\Users\parham\Desktop\tsai\Final\ts_train400_multi.ts', "w") as text_file:
    text_file.write(finaltxt)

In [11]:
#result.to_csv("same size.csv",index=False)